In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
import time
import os
#from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from sklearn.linear_model import LogisticRegression
#from skearn.neighbors import NearestNeighbors
#from sklearn.neighbors.nearest_centroid import NearestCentroid
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.neighbors import RadiusNeighborsClassifier
#from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
#from sklearn.tree import DecisionTreeClassifier
#import lightgbm as lgb
#from sklearn.multiclass import OneVsRestClassifier
from lightgbm import LGBMClassifier

In [ ]:
def assesed(value):
    if value == 'Not able to inspect':
        return 0
    elif value == 'Interior':
        return 1
    elif value == 'Exterior':
        return 2
    elif value == 'Both':
        return 3
    else:
        return 5

In [ ]:
def roof_type(value):
    if value == 'Bamboo/Timber-Light roof':
        return 5
    elif value == 'Bamboo/Timber-Heavy roof':
        return 3
    else:
        return 1

In [ ]:
def foundation_type(value):
    if value == 'Bamboo/Timber':
        return 5
    elif value == 'RC':
        return 3
    elif value == 'Mud mortar-Stone/Brick':
        return 2
    elif value == 'Cement-Stone/Brick':
        return 1
    else:
        return 0

In [ ]:
def ground_floor_type(value):
    if value == 'Mud':
        return 5
    elif value == 'Timber':
        return 4
    elif value == 'RC':
        return 3
    elif value == 'Brick/Stone':
        return 1
    else:
        return 0

In [ ]:
def land_surface_condition(value):
    if value == 'Steep slope':
        return 5
    elif value == 'Moderate slope':
        return 3
    else:
        return 1

In [ ]:
def other_floor_type(value):
    if value == 'Not applicable':
        return 4
    elif value == 'TImber/Bamboo-Mud':
        return 4
    elif value == 'Timber-Planck':
        return 3
    else:
        return 2

In [ ]:
def position(value):
    if value == 'Not attached':
        return 3
    else:
        return 4

In [ ]:
def plan_configuration(value):
    if value == 'Rectangular' or value == 'Square' or value == 'T-shape':
        return 2
    elif value == 'Multi-projected' or value == 'Building with Central Courtyard' or value == 'Others' or value == 'U-shape':
        return 3
    elif value == 'E-shape' or value == 'L-shape' or value == 'H-shape':
        return 4

In [ ]:
def condition_post_eq(value):
    if value == 'Damaged-Rubble Clear-New building built' or value == 'Covered by landslide' or value == 'Damaged-Rubble clear' or value == 'Damaged-Rubble unclear':
        return 5
    elif value == 'Damaged-Not used':
        return 4
    elif value == 'Damaged-Used in risk':
        return 3
    elif value == 'Damaged-Repaired and used':
        return 2
    else:
        return 1

In [ ]:
def make_mean(feature, alpha, in_mattr):
    global_mean = in_mattr['damage_grade'].mean()
    values = dict(in_mattr[feature].value_counts())
    items = list(values.keys())
    mean_target = {}
    for i in items:
        mean_target[i] = in_mattr[in_mattr[feature] == i]['damage_grade'].mean()
    out = {}
    for i in items:
        out[i] = (mean_target[i] * values[i] + global_mean * alpha) / (values[i] + alpha)
    return out

In [ ]:
def make_feature(feature, values, in_mattr):
    in_mattr[feature + '_mean'] = in_mattr[feature].map(lambda m: values[m])

In [ ]:
x = pd.read_csv('../Dataset/train.csv')

In [ ]:
building_structure = pd.read_csv('../Dataset/Building_Structure.csv')
building_structure.drop(['district_id', 'vdcmun_id'], axis=1, inplace=True)
x = pd.merge(x, building_structure, on=['building_id'])
del building_structure
x = x.set_index('building_id')
x['damage_grade'] = x['damage_grade'].map(lambda m:int(list(m.split(' '))[1]))

In [ ]:
y = x['damage_grade']
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.25, random_state=1)

In [ ]:
filler = Imputer(strategy='most_frequent')
x_train = filler.fit_transform(x_train)
x_validation = filler.fit_transform(x_validation)
x = preprocessing.normalize(x, norm='l2')

In [ ]:
columns = [
    'area_assesed', 'roof_type', 'foundation_type', 'ground_floor_type', 'land_surface_condition', 'other_floor_type', 'position', 'plan_configuration',
    'condition_post_eq', 'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
    'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction', 'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall', 'has_repair_started',
    'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_mud_mortar_brick', 
    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered', 
    'has_superstructure_rc_engineered', 'has_superstructure_other']

In [ ]:
feature_dict = {}
for i in columns:
    feature_dict[i] = make_mean(i, 3.5, x_train)

In [ ]:
for i in columns:
    make_feature(i, feature_dict[i], x_train)
    make_feature(i, feature_dict[i], x_validation)

In [ ]:
x_train['area_assesed'] = x_train['area_assesed'].map(assesed)
x_train['roof_type'] = x_train['roof_type'].map(roof_type)
x_train['foundation_type'] = x_train['foundation_type'].map(foundation_type)
x_train['ground_floor_type'] = x_train['ground_floor_type'].map(ground_floor_type)
x_train['land_surface_condition'] = x_train['land_surface_condition'].map(land_surface_condition)
x_train['other_floor_type'] = x_train['other_floor_type'].map(other_floor_type)
x_train['position'] = x_train['position'].map(position)
x_train['plan_configuration'] = x_train['plan_configuration'].map(plan_configuration)
x_train['condition_post_eq'] = x_train['condition_post_eq'].map(condition_post_eq)

In [ ]:
x_validation['area_assesed'] = x_validation['area_assesed'].map(assesed)
x_validation['roof_type'] = x_validation['roof_type'].map(roof_type)
x_validation['foundation_type'] = x_validation['foundation_type'].map(foundation_type)
x_validation['ground_floor_type'] = x_validation['ground_floor_type'].map(ground_floor_type)
x_validation['land_surface_condition'] = x_validation['land_surface_condition'].map(land_surface_condition)
x_validation['other_floor_type'] = x_validation['other_floor_type'].map(other_floor_type)
x_validation['position'] = x_validation['position'].map(position)
x_validation['plan_configuration'] = x_validation['plan_configuration'].map(plan_configuration)
x_validation['condition_post_eq'] = x_validation['condition_post_eq'].map(condition_post_eq)

In [ ]:
x_train.drop(['damage_grade'], axis=1, inplace=True)
x_validation.drop(['damage_grade'], axis=1, inplace=True)

In [ ]:
lgbm_1 = LGBMClassifier()
lgbm_2 = LGBMClassifier(learning_rate=0.05)
rf_1 = RandomForestClassifier(n_estimators=250, max_depth=27, n_jobs=-1, oob_score=True, verbose=1)
rf_2 = RandomForestClassifier(n_estimators=150, max_depth=20, n_jobs=-1, oob_score=True, verbose=1)
bg_1 = BaggingClassifier(n_estimators=50, n_jobs=-1, oob_score=True, verbose=1)
bg_2 = BaggingClassifier(n_estimators=100, n_jobs=-1, oob_score=True, verbose=1)
model = VotingClassifier(estimators=[('lgbm_1', lgbm_1), ('lgbm_2', lgbm_2), ('rf_1', rf_1), ('rf_2', rf_2), ('bg_1', bg_1), ('bg_2', bg_2), ], voting='hard', n_jobs=-1)
model.fit(x_train, y_train)

In [ ]:
print('Train Accuracy:', accuracy_score(y_train, model.predict(x_train)))
print('Validation Accuracy:', accuracy_score(y_validation, model.predict(x_validation)))

In [ ]:
pd.DataFrame(data={'res':model.predict(x_train)})['res'].value_counts()

In [ ]:
if hasattr(model, 'coef_'):
    df_importances = sorted(list(zip(x_train.columns, model.coef_.ravel())), key=lambda tpl: tpl[1], reverse=True)
else:
    df_importances= sorted(list(zip(x_train.columns, model.feature_importances_.ravel())), key=lambda tpl: tpl[1], reverse=True)
df_importances = pd.DataFrame(df_importances, columns=['feature', 'importance'])
df_importances = df_importances.set_index('feature')
df_importances.plot(kind='bar', figsize=(60, 10))
plt.show()

In [ ]:
del x_train
del x_validation
del y_train
del y_validation
del x
del y

In [ ]:
x_test = pd.read_csv('../Dataset/test.csv')

In [ ]:
building_structure = pd.read_csv('../Dataset/Building_Structure.csv')
building_structure.drop(['district_id', 'vdcmun_id'], axis=1, inplace=True)
x_test = pd.merge(x_test, building_structure, on=['building_id'])
del building_structure
n_index = x_test['building_id']
x_test = x_test.set_index('building_id')
x_test = x_test.fillna(0)

In [ ]:
for i in columns:
    make_feature(i, feature_dict[i], x_test)

In [ ]:
x_test['area_assesed'] = x_test['area_assesed'].map(assesed)
x_test['roof_type'] = x_test['roof_type'].map(roof_type)
x_test['foundation_type'] = x_test['foundation_type'].map(foundation_type)
x_test['ground_floor_type'] = x_test['ground_floor_type'].map(ground_floor_type)
x_test['land_surface_condition'] = x_test['land_surface_condition'].map(land_surface_condition)
x_test['other_floor_type'] = x_test['other_floor_type'].map(other_floor_type)
x_test['position'] = x_test['position'].map(position)
x_test['plan_configuration'] = x_test['plan_configuration'].map(plan_configuration)
x_test['condition_post_eq'] = x_test['condition_post_eq'].map(condition_post_eq)

In [ ]:
prediction = model.predict(x_test)
prediction = np.transpose(prediction)

In [ ]:
y_submission = pd.DataFrame(data={'building_id': n_index, 'damage_grade': prediction})
y_submission['damage_grade'] = y_submission['damage_grade'].map(lambda x: 'Grade ' + str(x))
y_submission = y_submission.set_index('building_id')

In [ ]:
current_timestamp = int(time.time())
submission_path = 'submissions/{}.csv'.format(current_timestamp)
if not os.path.exists('submissions'):
    os.makedirs('submissions')
print(submission_path)
y_submission.to_csv(submission_path, index=True)